In [ ]:
"""
run anaconda prompt 
"""

import os

home_path = os.path.expanduser('~')
cur_path = os.getcwd()
conda_path = home_path + "\\anaconda3"
conda_script_path = home_path + "\\anaconda3\\Scripts\\activate.bat"
exc = ' '.join(['start', '%windir%\System32\cmd.exe "/K"', conda_script_path, conda_path])
!$exc

"""
run tensorboard server

conda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs
http://localhost:6006/
"""

In [2]:
import os

from manifest import Manifest
from logger import Logger
from remover import Remover

# 자동완성용 import
from env import Env
from runner import RunnerParams
from trainer import Trainer
from player import Player

from algorithms.dqn import DQN, DQNParams
from algorithms.dqn_runner import DQNRunner
from algorithms.ddqn import DDQN, DDQNParams
from algorithms.ddqn_runner import DDQNRunner
from algorithms.reinforce import Reinforce, ReinforceParams
from algorithms.reinforce_runner import ReinforceRunner
from algorithms.actorcritic import ActorCritic, ActorCriticParams
from algorithms.actorcritic_runner import ActorCriticRunner


import collections

def train():
    cases = Trainer().allcases
    cartpole_cases = filter(lambda x:x[0] == Env.CARTPOLE, cases)
    lunar_cases = filter(lambda x:x[0] == Env.LUNARLANDER, cases)

    check_interval_arr = [5]
    tr = Trainer(check_interval_arr)
    for env, algo in lunar_cases:
        tr.add_case(env, algo)
    tr.run()
    
    check_interval_arr = [10]
    tr = Trainer(check_interval_arr)
    for env, algo in cartpole_cases:
        tr.add_case(env, algo)
    tr.run()
    

def play():
    PATH = os.path.join(os.getcwd(), 'weights')
    filenames = ['.'.join(tokens[:-1]) 
                        for name in os.listdir(PATH) 
                        if (tokens := name.split('.')) and len(tokens) > 2]

    Player('weights', filenames).run(runner_param_dic={'max_episode': 125})


def train_dqn_ddqn():
    cases = Trainer().allcases
    target_cases = filter(lambda x:(x[1] == DQN) or (x[1] == DDQN), cases)
    cartpole_cases = filter(lambda x:x[0] == Env.CARTPOLE, target_cases)
    lunar_cases = filter(lambda x:x[0] == Env.LUNARLANDER, target_cases)

    check_interval_arr = [5]
    tr = Trainer(check_interval_arr)
    for env, algo in lunar_cases:
        tr.add_case(env, algo)
    tr.run()
    
    # check_interval_arr = [10]
    # tr = Trainer(check_interval_arr)
    # for env, algo in cartpole_cases:
    #     tr.add_case(env, algo)
    # tr.run()

for i in range(10):
    train_dqn_ddqn()

# Remover().remove_dirs(['runs', 'weights', 'videos'])
# Remover().remove_dirs(['logs'])

# for i in range(10):
#     train()

# print_interval=10, max_epi=100 > 4분 35초
# print_interval=0, max_epi=100 > 4분 25초
# save_check_log=True, print_interval=0, max_epi=100 > 4분 37초

# for _ in range(10):
#     replay()

초기 설정
algorithm: DQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
시뮬레이션 시작
종료 조건 만족. 최종 5번 평균 점수 212.38898994006377
시뮬레이션 종료, 네트워크 저장
초기 설정
algorithm: DDQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
시뮬레이션 시작
종료 조건 만족. 최종 5번 평균 점수 213.32533953809516
시뮬레이션 종료, 네트워크 저장
초기 설정
algorithm: DQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
시뮬레이션 시작
종료 조건 만족. 최종 5번 평균 점수 205.03419902320607
시뮬레이션 종료, 네트워크 저장
초기 설정
algorithm: DDQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
시뮬레이션 시작
종료 조건 만족. 최종 5번 평균 점수 206.2943887187817
시뮬레이션 종료, 네트워크 저장
초기 설정
algorithm: DQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
시뮬레이션 시작
종료 조건 만족. 최종 5번 평균 점수 236.43637720242063
시뮬레이션 종료, 네트워크 저장
초기 설정
algorithm: DDQN
env: LunarLander-v2
state space: (8,)
action space: Discrete(4)
시뮬레이션 시작
종료 조건 만족. 최종 5번 평균 점수 210.9443631407214
시뮬레이션 종료, 네트워크 저장
초기 설정
algorithm: DQN
env: LunarLander-v2
state space: (8,)
action space: Discrete

In [ ]:
def tuning_reinforce():
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, Reinforce)]:
        start = 5
        k = 5
        for i in range(21):
            hparam = tr.default_hyperparam(env, algo)
            hparam.learning_rate = (start + k*i)/10000
            tr.add_case(env, algo, hparam)

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_actorcritic():
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, ActorCritic)]:
        start = 1
        k = 5
        for i in range(21):
            hparam = tr.default_hyperparam(env, algo)
            hparam.learning_rate = (start + k*i)/10000
            tr.add_case(env, algo, hparam)

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_dqn_node_cartpole():
    """
    8
    """
    tr = Trainer([20])
    for env, algo in [(Env.CARTPOLE, DQN)]:
        n_node = 2
        for i in range(8):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_dqn_node_lunarlander():
    """
    64~256
    """
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, DQN)]:
        n_node = 2
        for i in range(8):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_ddqn_node_cartpole():
    """
    8~16
    """
    tr = Trainer([20])
    for env, algo in [(Env.CARTPOLE, DDQN)]:
        n_node = 64
        for i in range(2):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_ddqn_node_lunarlander():
    """
    32, 128, 256
    """
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, DDQN)]:
        n_node = 32
        for i in range(4):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

# Remover().remove_dirs(['runs', 'weights', 'videos'])
# tuning_dqn_node_cartpole()
# tuning_dqn_node_lunarlander()
tuning_ddqn_node_lunarlander()